In [3]:
from math import sqrt
from collections import namedtuple

Point = namedtuple("Point", ['x', 'y'])
Facility = namedtuple("Facility", ['index', 'setup_cost', 'capacity', 'location'])
Customer = namedtuple("Customer", ['index', 'demand', 'location'])


def dist(p1, p2):
    return sqrt((p1.x-p2.x)**2 + (p1.y-p2.y)**2)


def parse_input(input_data):
    # parse the input
    lines = input_data.split('\n')

    parts = lines[0].split()
    facility_count = int(parts[0])
    customer_count = int(parts[1])

    facilities = []
    for i in range(1, facility_count+1):
        parts = lines[i].split()
        facilities.append(Facility(i-1, float(parts[0]), int(parts[1]), Point(float(parts[2]), float(parts[3])) ))

    customers = []
    for i in range(facility_count+1, facility_count+1+customer_count):
        parts = lines[i].split()
        customers.append(Customer(i-1-facility_count, int(parts[0]), Point(float(parts[1]), float(parts[2]))))

    return customers, facilities


%pip install pulp

In [4]:
import logging

class SolverBasic:
    def __init__(self):
        self.facilities = []
        self.customers = []
        self.logger = logging.getLogger('facility_planner')
        self.logger.setLevel(logging.DEBUG)
        self.log_file_name = 'solver.log'
        fh = logging.FileHandler(self.log_file_name, mode='w')
        fh.setLevel(logging.DEBUG)
        ch = logging.StreamHandler()
        ch.setLevel(logging.ERROR)
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        fh.setFormatter(formatter)
        ch.setFormatter(formatter)
        self.logger.addHandler(fh)
        self.logger.addHandler(ch)
        pass

    def solve(self, facilities, customers):
        # build a trivial solution
        # pack the facilities one by one until all the customers are served
        solution = [-1]*len(customers)
        capacity_remaining = [f.capacity for f in facilities]

        facility_index = 0
        for customer in customers:
            if capacity_remaining[facility_index] >= customer.demand:
                solution[customer.index] = facility_index
                capacity_remaining[facility_index] -= customer.demand
            else:
                facility_index += 1
                assert capacity_remaining[facility_index] >= customer.demand
                solution[customer.index] = facility_index
                capacity_remaining[facility_index] -= customer.demand

        used = [0]*len(facilities)
        for facility_index in solution:
            used[facility_index] = 1

        # calculate the cost of the solution
        obj = sum([f.setup_cost*used[f.index] for f in facilities])
        for customer in customers:
            obj += dist(customer.location, facilities[solution[customer.index]].location)

        return obj, solution


In [5]:
class SolverGreedy(SolverBasic):
    def init_model(self):
        pass

    def solve(self, facilities, customers):
        self.facilities = facilities
        self.customers = customers
        self.init_model()
        # TODO implement greedy algorithm
        pass


In [6]:
import pulp

class SolverPULP(SolverGreedy):
    def __init__(self):
        super().__init__()
        self.distances = []
        self.c2f = []  # customer to facility assignement
        self.f_d = []  # facility demands
        self.customers = []
        self.customers_count = 0
        self.facilities = []
        self.facilities_count = 0


    def calculate_dist(self, customer, facility):
        return dist(customer.location, facility.location)

    def calculate_dists(self):
        fc = len(self.facilities)
        cc = len(self.customers)
        self.distances = [[0]*fc for _ in range(cc)]
        for c in range(cc):
            for f in range(fc):
                self.distances[c][f] = self.calculate_dist(self.customers[c], self.facilities[f])
        # self.logger.debug(f"distances: {self.distances}")

    def input_feasibility_check(self):
        total_capacity = 0
        for f in self.facilities:
            total_capacity += f.capacity
        total_demand = 0
        for c in self.customers:
            total_demand += c.demand
        assert total_demand < total_capacity
        self.logger.debug(f"demand:{total_demand}, capacity:{total_capacity}")

    def output_feasibility_check(self):
        pass

    def init_model(self):
        # customers to facilities assignement
        if not self.facilities or not self.customers:
            return  # empty input, nothing to do

        self.customers_count = len(self.customers)
        self.facilities_count = len(self.facilities)

        self.input_feasibility_check()

        # customer to facility assignement martix
        self.c2f = [
            [
                pulp.LpVariable(
                    f"c{cn}_f{fn}", cat='Binary'
                    ) for fn, _ in enumerate(self.facilities)
            ] for cn, _ in enumerate(self.customers)
        ]
        # self.logger.debug(self.c2f)
        
        self.calculate_dists()
        # facility demand
        self.f_d = [
            pulp.LpVariable(
                f"f_d_{fn}", cat=pulp.LpContinuous, lowBound=0, upBound=f.capacity
            ) for fn, f in enumerate(self.facilities)
        ]
        
        # facility open binary
        self.f_o = [
            pulp.LpVariable(
                f"f_o_{fn}", cat=pulp.LpBinary
            ) for fn, _ in enumerate(self.facilities)
        ]
        # self.logger.debug(self.f_d)
        
        
        self.model = pulp.LpProblem("Cost_minimising_planning_problem", pulp.LpMinimize)
        
        # === objective function ===
        # sum of all distances to the open centre
        # add setup costs
        self.model += pulp.lpSum(
            [
                self.distances[cn][fn] * self.c2f[cn][fn]
                for cn in range(self.customers_count)
                for fn, f in enumerate(self.facilities)
            ] + [
                f.setup_cost * self.f_o[fn]
                for fn, f in enumerate(self.facilities)
            ]
        )
        # self.logger.debug(f"model: {self.model}")
        
        # === constraints definition ===
        
        if True:
            # set f_o if sum of assignments to it > 0
            sum_delta = []
        
            for fn in range(self.facilities_count):
                sum_a = 0
                for cn in range(self.customers_count):
                    sum_a += self.c2f[cn][fn]
                # condition if sum_a > 0 then set f_o to 1
                # https://math.stackexchange.com/questions/2500415/how-to-write-if-else-statement-in-linear-programming/2501007
                sum_delta.append(pulp.LpVariable(f"sum_delta{fn}", cat=pulp.LpBinary))
                self.model += sum_a >= - self.customers_count * (1-sum_delta[-1])
                self.model += sum_a <= self.customers_count * sum_delta[-1]
                self.model += 1 - self.customers_count*(1-sum_delta[-1]) <= self.f_o[fn]
                self.model += 1 + self.customers_count*(1-sum_delta[-1]) >= self.f_o[fn]
                self.model += - self.customers_count*sum_delta[-1] <= self.f_o[fn]
                self.model += self.customers_count*sum_delta[-1] >= self.f_o[fn]
        self.logger.debug(f"model: {self.model}")

        # sum demands <= facility capacity
        for fn,f in enumerate(self.facilities):
            sum_d = 0
            for cn, c in enumerate(self.customers):
                sum_d += c.demand * self.c2f[cn][fn]
            self.model += self.f_d[fn] == sum_d
            self.model += self.f_d[fn] <= f.capacity

        # each customer must be assigned to just one facility
        for cn in range(self.customers_count):
            self.model += pulp.lpSum([self.c2f[cn]]) == 1

        # should be not needed
        if False:
            for fn in range(self.facilities_count):
                sum_a = 0
                for cn in range(self.customers_count):
                    sum_a += self.c2f[cn][fn]
                self.model += sum_a <= self.customers_count
                    
    def process_results(self):
        # self.logger.debug(self.f_o)
        # self.logger.debug(self.c2f)
        obj = pulp.value(self.model.objective)
        self.logger.debug(f"objective val {obj}")
        # show which facilities are open
        for fn in range(self.facilities_count):
            f_o = pulp.value(self.f_o[fn])
            if f_o:
                self.logger.debug(f"f_o:{fn}=={f_o}")
            
        # show assignments of customers to facilities
        for cn in range(self.customers_count):
            for fn in range(self.facilities_count):
                assigned = pulp.value(self.c2f[cn][fn])
                if assigned:
                    self.logger.debug(f"c{cn}->f{fn}")
                
        # show total demand from each open facility from the solver
        for fn in range(self.facilities_count):
            f_d = pulp.value(self.f_d[fn])
            if f_d:
                self.logger.debug(f"f_d:{fn}={f_d}")
        # recalculate total demand based on assignments
        total_demand = 0
        for fn, f in enumerate(self.facilities):
            f_demand = 0
            for cn, c in enumerate(self.customers):
                assigned = pulp.value(self.c2f[cn][fn])
                if assigned == 1:
                    f_demand += c.demand
            total_demand += f_demand
                    # self.logger.debug(f"c{cn}f{fn}, demand={tdemand={total_demand}")
            self.logger.debug(f"f{fn} capacity={f.capacity}, demand={f_demand}")
        self.logger.debug(f"total demand={total_demand}")

        solution = []
        return obj, solution

    def solve(self, facilities, customers):
        self.facilities = facilities
        self.customers = customers
        self.model = None
        self.init_model()
        # TODO: implement PULP-based algorithm
        self.model.solve()
        self.logger.info(f"solving status: {pulp.LpStatus[self.model.status]}")
        self.process_results()
        obj = 0
        solution = []
        return obj, solution


In [7]:
def solve_it(input_data):
    # Modify this code to run your optimization algorithm

    # parse the input
    customers, facilities = parse_input(input_data)

    solver = SolverPULP()
    obj, solution = solver.solve(facilities, customers)

    # prepare the solution in the specified output format
    output_data = '%.2f' % obj + ' ' + str(0) + '\n'
    output_data += ' '.join(map(str, solution))

    return output_data



In [8]:
input_file = "data/facility_planning/fl_16_1"

In [9]:
with open(input_file, 'r') as input_data_file:
    input_data = input_data_file.read()
print(solve_it(input_data))


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/8dd06fb2246749b3b3941c12cc981622-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/8dd06fb2246749b3b3941c12cc981622-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 183 COLUMNS
At line 6055 RHS
At line 6234 BOUNDS
At line 7083 ENDATA
Problem MODEL has 178 rows, 848 columns and 3392 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3.36246e+06 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 0 strengthened rows, 30 substitutions
Cgl0004I processed model has 82 rows, 816 columns (816 integer (816 of which binary)) and 2416 elements
Cbc0038I Initial state - 24 integers unsatisfied sum - 4.18719
Cbc0038I Pass   1: suminf.    3.24859 (20) obj. 3.43149e+06 iteration